<a href="https://colab.research.google.com/github/MOFU0712/it_news_check_and_analysis/blob/main/news_check_in_notion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install anthropic
!pip install notion_client

## ニュースページの要約

In [ ]:
import os
from google.colab import userdata
from notion_client import Client
from anthropic import Anthropic

# NotionとAnthropicのAPI設定
NOTION_TOKEN = userdata.get('NOTION_API_KEY')
NOTION_DATABASE_ID = userdata.get('PICKUP_DATABASE_KEY')
ANTHROPIC_API_KEY = userdata.get('CLAUDE_API_KEY')

# NotionとAnthropicのクライアント初期化
notion = Client(auth=NOTION_TOKEN)
anthropic = Anthropic(api_key=ANTHROPIC_API_KEY)

def get_page_content(page_id):
    """ページの本文コンテンツを取得する"""
    blocks = notion.blocks.children.list(block_id=page_id)
    content = ""

    for block in blocks["results"]:
        if block["type"] == "paragraph":
            try:
                content += block["paragraph"]["rich_text"][0]["text"]["content"] + "\n"
            except (KeyError, IndexError):
                continue

    return content

def extract_text_from_response(response):
    """APIレスポンスからテキスト部分のみを抽出する"""
    if hasattr(response, 'text'):
        return response.text
    elif isinstance(response, list) and len(response) > 0:
        first_block = response[0]
        if hasattr(first_block, 'text'):
            return first_block.text
    return str(response)

def generate_summary(content):
    """Claude 3.5 Haikuを使用して要約を生成する"""
    if not content.strip():
        return ""

    try:
        message = anthropic.messages.create(
            model="claude-3-haiku-20240307",
            # model='claude-3-7-sonnet-20250219',
            max_tokens=300,
            messages=[{
                "role": "user",
                "content": f"以下の文章をニュースまとめ記事用に、**必ず日本語**で、2文程度で簡潔に要約してください。箇条書きではなく、文章として書き、文体はですます調で、出力は要約文だけでお願いします。：\n\n{content}"
            }]
        )

        # メッセージの内容からテキスト部分のみを抽出
        content = message.content
        summary = extract_text_from_response(content)

        return summary
    except Exception as e:
        print(f"Error generating summary: {e}")
        return ""

def update_empty_summaries():
    """summaryが空のページを検索し、要約を生成して更新する"""
    try:
        # データベースの全ページを取得
        pages = notion.databases.query(
            database_id=NOTION_DATABASE_ID,
            filter={
                "property": "summary",
                "rich_text": {
                    "is_empty": True
                }
            }
        )

        for page in pages["results"]:
            try:
                page_id = page["id"]

                # ページの本文を取得
                content = get_page_content(page_id)

                # 要約を生成
                summary = generate_summary(content)

                if summary:
                    # summaryプロパティを更新
                    notion.pages.update(
                        page_id=page_id,
                        properties={
                            "summary": {
                                "rich_text": [{
                                    "type": "text",
                                    "text": {
                                        "content": summary
                                    }
                                }]
                            }
                        }
                    )
                    print(f"Updated summary for page {page_id}")
            except Exception as e:
                print(f"Error processing page {page_id}: {e}")
                continue

    except Exception as e:
        print(f"Error querying database: {e}")

if __name__ == "__main__":
    update_empty_summaries()

Updated summary for page 1d96a7ec-2649-8159-9bc5-e0ac365790b4
Updated summary for page 1d96a7ec-2649-8104-8143-e5021b900964
Updated summary for page 1d96a7ec-2649-81eb-a656-cfb7a458f4e1


In [3]:
import requests
import os
from datetime import datetime, timedelta, date
from google.colab import userdata
import anthropic
import re
from typing import List, Tuple
from IPython.display import Markdown, display


def pick_daily_news_from_database(url, headers):


# データベースからデータを取得するためのフィルター
    data = {
        "filter": {
            "and": [
                {
                    "property": "flag",
                    "status": {
                        "equals": "pick"
                    }
                },
                {
                    "or": [
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "day"
                            }
                        },
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "week"
                            }
                        },
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "month"
                            }
                        }
                    ]
                }
            ]
        }
    }

    # APIリクエストを送信
    response = requests.post(url, headers=headers, json=data)
    results = response.json()
    return results

def pick_weekly_news_from_database(url, headers):


# データベースからデータを取得するためのフィルター
    data = {
        "filter": {
            "and": [
                {
                    "property": "flag",
                    "status": {
                        "equals": "pick"
                    }
                },
                {
                    "or": [
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "week"
                            }
                        },
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "month"
                            }
                        }
                    ]
                }
            ]
        }
    }

    # APIリクエストを送信
    response = requests.post(url, headers=headers, json=data)
    results = response.json()
    return results

def pick_monthly_news_from_database(url, headers):


    # データベースからデータを取得するためのフィルター
    data = {
        "filter": {
            "and": [
                {
                    "property": "flag",
                    "status": {
                        "equals": "pick"
                    }
                },
                {
                    "property": "pickup_type",
                    "select": {
                        "equals": "month"
                    }
                }
            ]
        }

    }


    # APIリクエストを送信
    response = requests.post(url, headers=headers, json=data)
    results = response.json()
    return results

def add_page_to_database(page_title, database_key, news_contents):

    NOTION_API_KEY = userdata.get('NOTION_API_KEY')
    DATABASE_ID = userdata.get(database_key)

    url = 'https://api.notion.com/v1/pages'

    headers =  {
        'Notion-Version': '2022-06-28',
        'Authorization': 'Bearer ' + NOTION_API_KEY,
        'Content-Type': 'application/json',
    }

    json_data = {
        'parent': { 'database_id': DATABASE_ID },
        'properties': {
            'name': {
                'title': [
                    {
                        'text': {
                            'content': page_title
                        }
                    }
                ]
            },
        },
    }

    page_response = requests.post('https://api.notion.com/v1/pages', headers=headers, json=json_data)
    new_page_id = page_response.json()['id']  # 新しく作成されたページのIDを取得
    return new_page_id

def add_content_to_page(headers, new_page_id, text):


    # ページにブロック（テキスト）を追加
    block_data = {
        "children": [
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": text
                            }
                        }
                    ]
                }
            }
        ]
    }

    block_response = requests.patch(f'https://api.notion.com/v1/blocks/{new_page_id}/children', headers=headers, json=block_data)

def split_text(long_text, length=2000):
    return [long_text[i:i+length] for i in range(0, len(long_text), length)]

def main(pickup_type, title_date, title_name):
    NOTION_API_KEY = userdata.get('NOTION_API_KEY')
    DATABASE_ID = userdata.get('PICKUP_DATABASE_KEY')

    url = f'https://api.notion.com/v1/databases/{DATABASE_ID}/query'

    headers =  {
        'Notion-Version': '2022-06-28',
        'Authorization': 'Bearer ' + NOTION_API_KEY,
        'Content-Type': 'application/json',
    }

    today = date.today()
    formatted_date = today.strftime("%Y-%m-%d")

    if pickup_type == "day":
        update_database_key = "DAILY_DATABASE_KEY"
        results = pick_daily_news_from_database(url, headers)
        page_title_header = "daily_news"
    elif pickup_type == "week":
        update_database_key = "WEEKLY_DATABASE_KEY"
        results = pick_weekly_news_from_database(url, headers)
        page_title_header = "weekly_news"
    elif pickup_type == "month":
        update_database_key = "MONTHLY_DATABASE_KEY"
        results = pick_monthly_news_from_database(url, headers)
        page_title_header = "monthly_news"

    contents_list = []
    for result in results["results"]:
        properties = result["properties"]
        news_url = properties["URL"]["url"]
        tag = properties["tag"]["select"]["name"]
        if not properties["summary"]["rich_text"]:
            print("summaryが空です")
            abstract = ""
        else:
            abstract = properties["summary"]["rich_text"][0]["text"]["content"]
        title = properties["name"]["title"][0]["text"]["content"]
        content = [tag, title, news_url, abstract]
        contents_list.append(content)

    sorted_contents_list = sorted(contents_list, key=lambda x: x[0], reverse=True)

    news_contents = ""
    for content in sorted_contents_list:
        tag = content[0]
        title = content[1]
        news_url = content[2]
        abstract = content[3]

        news_contents += f"\n【{tag}】 {title} {news_url} \n {abstract} \n --------------------------------"


    page_title = f"{page_title_header}{title_date}{title_name}"


    new_page_id = add_page_to_database(page_title, update_database_key, news_contents)

    contents = split_text(news_contents)

    for content in contents:
        add_content_to_page(headers, new_page_id, content)

    return news_contents

def split_news_articles(text: str) -> List[str]:
  """
  入力テキストを個別のニュース記事に分割する

  Args:
      text (str): 入力テキスト

  Returns:
      List[str]: 個別のニュース記事のリスト
  """
  articles = text.split('-----------------------------')
  return [article.strip() for article in articles if article.strip()]

def process_individual_news(client: anthropic.Client, article: str) -> str:
    """
    個別のニュース記事を処理する

    Args:
        client: Anthropic APIクライアント
        article (str): 1つのニュース記事

    Returns:
        str: 処理済みの記事（タイトルと要約）
    """
    system_prompt = """
    以下のニュース記事を処理してください：
    1. 20語程度の一文でタイトルを作成
    2. 200文字程度、2-3文で要約
    3. 以下の形式で出力：

    [OUTPUT_START]
    作成したニュースタイトル
    【タグ】ニュースの要約文 ニュースのURL
    [OUTPUT_END]

    ・元テキストの【】タグをそのまま使用
    ・ですます調で記述
    """

    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0,
        system=system_prompt,
        messages=[{"role": "user", "content": article}]
    )

    response = message.content[0].text

    # 出力を抽出
    start_idx = response.find('[OUTPUT_START]') + len('[OUTPUT_START]')
    end_idx = response.find('[OUTPUT_END]')
    return response[start_idx:end_idx].strip()

def generate_title_and_impression(client: anthropic.Client, summaries: str) -> Tuple[str, str]:
    """
    全ての要約からタイトルと感想を生成

    Args:
        client: Anthropic APIクライアント
        summaries (str): 全ての記事の要約

    Returns:
        Tuple[str, str]: (総合タイトル, 感想)
    """
    system_prompt = """
    以下のニュース要約から、総合タイトルと感想を生成してください。

    タイトル:
    ・形式：「[主要ニュースのポイントをカンマ区切りで6つ程度]｜[月日]のIT・AIニュースピックアップ！」
    ・重要ニュース、インパクトの大きな出来事、新しい潮流を優先
    ・モデル名や製品、サービス名はかぎかっこ書きで必ず入れて下さい
    ・OpenAI, Google, Anthropic, Microsoft, Amazon, ByteDance,Alibaba,Sakana AIのニュースは必ず入れてください


    感想:
    ・タイトルで選んだキーワードを盛り込む
    ・タイトルに選んだニュースの詳細を記述（要約文を加工）
    ・ニュースの詳細のあとに、未来の展望・将来予想のような感想を記述
    ・ニュースはひとつひとつ感想を記載（まとめない）
    ・親しみやすく優しいが丁寧な人柄が感じられる文体
    ・1000文字程度

    必ず以下の形式で出力してください：
    [TITLE_START]
    （総合タイトル）
    [TITLE_END]

    [IMPRESSION_START]
    （ニュースの感想）
    [IMPRESSION_END]
    """

    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=2000,
        temperature=0,
        system=system_prompt,
        messages=[{"role": "user", "content": summaries}]
    )

    response = message.content[0].text

    # タイトルと感想を抽出
    title = response[response.find('[TITLE_START]')+len('[TITLE_START]'):response.find('[TITLE_END]')].strip()
    impression = response[response.find('[IMPRESSION_START]')+len('[IMPRESSION_START]'):response.find('[IMPRESSION_END]')].strip()

    return title, impression

def process_all_news(input_text: str) -> Tuple[str, str, str]:
    """
    全てのニュースを処理する

    Args:
        input_text (str): 入力テキスト

    Returns:
        Tuple[str, str, str]: (ニュースまとめ, 総合タイトル, 感想)
    """
    api_key = userdata.get('CLAUDE_API_KEY')
    client = anthropic.Client(api_key=api_key)

    # ニュースを分割
    articles = split_news_articles(input_text)

    # 各ニュースを個別に処理
    processed_articles = []
    for article in articles:
        try:
            processed = process_individual_news(client, article)
            processed_articles.append(processed)
        except Exception as e:
            print(f"記事の処理中にエラーが発生: {str(e)}")

    # 全ての処理済み記事を結合
    all_summaries = "\n\n".join(processed_articles)

    # タイトルと感想を生成
    title, impression = generate_title_and_impression(client, all_summaries)

    return all_summaries, title, impression

def format_news_articles_for_note(text):
    """
    ニュース記事のテキストをnoteのエディタ向けに整形する関数
    様々なタグに対応し、完全に独立したブロックを作成します

    Args:
        text (str): 複数のニュース記事を含む入力テキスト

    Returns:
        str: note用に整形されたニュース記事テキスト
    """
    # システムの改行コードを使用
    newline = os.linesep

    # URLで記事を分割するパターン
    # URLの後に改行または文字列終端があるものを検出
    pattern = r'(.*?https://[^\s]+)(?:\n|$)'

    # 記事を抽出
    articles = re.findall(pattern, text, re.DOTALL)

    formatted_articles = []
    for article in articles:
        # URLを抽出
        url_pattern = r'(https://[^\s]+)$'
        url_match = re.search(url_pattern, article)
        if not url_match:
            continue

        url = url_match.group(1)

        # URLを除いた部分を取得
        content = article[:url_match.start()].strip()

        # タグパターンを検出（【...】の形式）
        tag_pattern = r'【[^】]+】'

        # 最初のタグの位置を検索
        tag_matches = list(re.finditer(tag_pattern, content))
        if not tag_matches:
            continue

        # タイトルは最初のタグの前まで
        first_tag_pos = tag_matches[0].start()
        title = content[:first_tag_pos].strip()

        # タグと説明文を抽出
        tag_content = content[first_tag_pos:].strip()

        # 段落区切りを使用して整形
        formatted_parts = [
            f"### {title}",
            tag_content,
            url
        ]

        # 各パートを段落区切りで結合
        formatted_article = f"{newline}{newline}".join(formatted_parts)
        formatted_articles.append(formatted_article)

    # 記事間により大きな区切りを入れる
    article_separator = f"{newline}{newline}{newline}"
    return article_separator.join(formatted_articles)

def display_news_markdown(news_title, news_impression, formatted_result):
    """
    ニュース関連の出力をマークダウン形式で表示する

    Args:
        news_title (str): ニュースのタイトル
        news_impression (str): ニュースの感想
        formatted_result (str): フォーマット済みのニュース本文
    """
    markdown_content = f"""# {news_title}

こんにちは、MOFUです。

{news_impression}

{formatted_result}
"""
    display(Markdown(markdown_content))

from typing import List, Dict
from collections import defaultdict

def extract_news_from_results(results: dict) -> List[Dict]:
    """
    Notionのクエリ結果から必要な情報を抽出する

    Args:
        results (dict): Notionのクエリ結果

    Returns:
        List[Dict]: 抽出された記事情報のリスト
    """
    news_list = []
    for result in results["results"]:
        properties = result["properties"]

        # 必要な情報を抽出
        news_info = {
            'tag': properties["tag"]["select"]["name"],
            'title': properties["name"]["title"][0]["text"]["content"],
            'url': properties["URL"]["url"],
            'summary': "" if not properties["summary"]["rich_text"] else
                      properties["summary"]["rich_text"][0]["text"]["content"]
        }
        news_list.append(news_info)

    return news_list

def group_news_by_tag(news_list: List[Dict]) -> Dict[str, List[Dict]]:
    """
    ニュース記事をタグごとにグループ化する

    Args:
        news_list (List[Dict]): ニュース記事のリスト

    Returns:
        Dict[str, List[Dict]]: タグでグループ化されたニュース記事
    """
    grouped_news = defaultdict(list)
    for news in news_list:
        grouped_news[news['tag']].append(news)

    # タグでソート
    return dict(sorted(grouped_news.items()))

def format_news_output_for_monthly_summary(grouped_news: Dict[str, List[Dict]]) -> str:
    """
    グループ化されたニュースを指定された形式でフォーマットする

    Args:
        grouped_news (Dict[str, List[Dict]]): タグでグループ化されたニュース記事

    Returns:
        str: フォーマットされたニュース記事テキスト
    """
    output = []
    for tag, news_items in grouped_news.items():
        output.append(f"### 【{tag}】")
        for news in news_items:
            output.extend([
                f"#### 〇 {news['title']}",
                news['url'],
                news['summary'] if news['summary'] else "",
                ""  # 記事間の空行
            ])
        output.append("")  # タグセクション間の空行

    return "\n".join(output)

def main_for_monthly_report(url: str, headers: dict) -> str:
    """
    メイン処理関数

    Args:
        url (str): NotionのAPIエンドポイント
        headers (dict): APIリクエストヘッダー

    Returns:
        str: フォーマットされたニュース記事テキスト
    """
    # flagがpickの記事を取得
    results = pick_daily_news_from_database(url, headers)

    # 記事情報を抽出
    news_list = extract_news_from_results(results)

    # タグでグループ化
    grouped_news = group_news_by_tag(news_list)

    # 指定された形式でフォーマット
    formatted_output = format_news_output_for_monthly_summary(grouped_news)

    return formatted_output


## ニュースまとめ記事の作成

In [ ]:
pickup_type = "day"
title_date = "2025-04-18"
title_name = ""

# Notionデータベースからflag='true'でpickup_typeに合致するものを抽出
news_contents = main(pickup_type, title_date, title_name)

# Claude APIを使って、ニュースの要約・整形、ブログタイトル、ブログの感想を作成
news_summary, news_title, news_impression = process_all_news(news_contents)

# noteに貼り付ける用の整形
formatted_result = format_news_articles_for_note(news_summary)

# マークダウン形式で結果を出力（出力結果をnoteにコピペすればOK）
display_news_markdown(
    news_title=news_title,
    news_impression=news_impression,
    formatted_result=formatted_result
)

# 「Gemini 2.5 Flash」思考プロセス搭載、「FramePack」「Wan」動画生成AI進化、Huawei「CloudMatrix 384」NVIDIAを上回る性能、OpenAI「Codex CLI」公開、中国AIモデル米国に追随、PLaMoの日本語埋め込みモデル｜4月18日のIT・AIニュースピックアップ！

こんにちは、MOFUです。

Googleが開発者向けに「Gemini 2.5 Flash」を早期公開しました。このモデルは複雑な課題を分解して応答を計画できる思考プロセス機能を搭載しており、主要AIモデルと同等の性能を維持しながらもコンパクトで低コストな運用を実現しています。開発者は用途に応じて柔軟な設定ができるようになっており、AIの実用性と経済性を両立させる新たな選択肢として期待できそうですね。

動画生成AIの分野では「FramePack」という新手法が登場しました。入力コンテキストを一定の長さに圧縮する特徴により、動画の長さに関係なく一定の処理負荷で生成できるのが特徴です。13Bという大規模モデルでもノートPCのGPUで処理できる実用性の高さは、クリエイターの間で広く活用される可能性を秘めていますね。

さらに動画生成AIの新モデル「Wan」も発表されました。時空間の因果性を考慮した設計と特徴量キャッシュの導入により、大規模モデルではOpenAIの「Sora」を超える性能を実現する可能性があるとのこと。小規模モデルでもGPUメモリ使用量を抑えた効率的な設計を採用しており、動画生成AIの民主化が一歩進んだように感じます。

ハードウェア面では、Huaweiが新AIアーキテクチャ「CloudMatrix 384 Supernode」を発表しました。384個のAscend 910Cチップを搭載し、NVIDIAのGB200 NVL72と比べて1.7倍の性能を実現しているとのこと。ただし消費電力は3.9倍と効率面での課題も残されています。中国製AIハードウェアの急速な進化は、グローバルなAI開発競争をさらに加速させそうです。

OpenAIは自然言語でシェルコマンドを実行できるコーディングエージェント「Codex CLI」をオープンソースで公開しました。Node.js製のこのツールは、ユーザーの自然言語指示からシェルコマンドを生成・実行できる便利な機能を提供しています。プログラミングの敷居をさらに下げる取り組みとして、多くの開発者に歓迎されるでしょう。

スタンフォード大学のAIインデックス調査によると、中国と米国の主要なAIモデルの性能差が大幅に縮小し、ほぼ同等の水準に達していることが明らかになりました。2024年の注目すべきAIモデル開発件数では、米国が40件、中国が15件を記録。AI開発における二大国の競争は今後も続き、技術革新のペースがさらに加速しそうです。

国内では、Preferred Networksが日本語に特化したローカル実行可能な新テキスト埋め込みモデル「PLaMo-Embedding-1B」を公開しました。ローカル環境で動作し、高性能な日本語テキストの埋め込みが可能な特徴を持ち、工場内でのデータ検索など、セキュアな環境での活用が期待されています。日本語処理に特化したAIモデルの充実は、国内企業のAI活用を大きく後押しするでしょう。

### Google、思考プロセスを備えた新型AI「Gemini 2.5 Flash」を開発者向けに早期公開

【新しいLLM】Googleが新たに公開したGemini 2.5 Flashは、複雑な課題を分解して応答を計画できる思考プロセス機能を搭載しています。この新機能により、他の主要なAIモデルと同等の性能を維持しながら、よりコンパクトで低コストな運用を実現しました。開発者は用途に応じて柔軟な設定が可能となっています。

https://developers.googleblog.com/en/start-building-with-gemini-25-flash/


### MongoDBとMCPの組み合わせが従来のRAG方式を超える可能性を示唆する新技術動向

【技術解説】MongoDBとMCPサーバーを組み合わせることで、JSON形式のデータに対してナチュラルな質問による効率的な検索が可能になっています。ただし、大量データの検索時にはトークン消費の増加やクライアントのフリーズに注意が必要です。この技術の発展により、従来のRAG方式に代わる新しいデータ検索手法として期待が高まっています。

https://zenn.dev/wooheum/articles/54adee2a91c26a


### OpenAI、自然言語でシェルコマンドを実行できるコーディングエージェント「Codex CLI」をオープンソースで公開

【技術解説】OpenAIが新たにリリースしたCodex CLIは、Node.js製のコーディングエージェントツールで、ユーザーの自然言語指示からシェルコマンドを生成・実行できます。独自のMarkdown風DIFF書式を採用し、OpenAI Responses APIと連携してソースコードの編集を可能にしています。

https://blog.lai.so/openai-codex/


### 動画生成AIの新手法「FramePack」が登場、ノートPCでも効率的な動画生成が可能に

【動画生成AI】効率的な次フレーム予測の新しいニューラルネットワーク構造「FramePack」が公開されました。入力コンテキストを一定の長さに圧縮する特徴により、動画の長さに関係なく一定の処理負荷で動画を生成できます。13Bという大規模なモデルでもラップトップのGPUで処理が可能な実用性の高い技術です。

https://github.com/lllyasviel/FramePack


### 動画生成AIの新モデル「Wan」が登場、Soraを上回る性能と効率的な設計を実現

【動画生成AI】新しい大規模ビデオ生成モデル「Wan」が発表されました。時空間の因果性を考慮した設計と特徴量キャッシュの導入により、大規模モデルではOpenAIのSoraを超える性能を実現する可能性があります。また、小規模モデルでもGPUメモリ使用量を抑えた効率的な設計を採用しており、実用性の高いモデルとなっています。

https://blog.shikoan.com/wan/?utm_source=feedly&utm_medium=rss&utm_campaign=wan


### デジタル庁の自動配送ロボット実証実験で安全課題が浮上、エレベーター挟まれ事故も発生

【ロボット・ドローン】デジタル庁が実施した自動配送ロボットの実証実験において、ロボット同士の接触やエレベーターでの挟まれ事故などの安全上の課題が明らかになりました。この結果を受けて、複数ロボットの安全な走行のための事業者間協調と共通ルールの整備を進めることが決定し、ロボットフレンドリー施設推進機構との連携も予定されています。

https://jidounten-lab.com/u_53902


### Huaweiが新AIアーキテクチャ「CloudMatrix 384」を発表、384個のチップ搭載でNVIDIA製品を性能で上回る

【ハードウェア】Huaweiが新たに発表したAIインフラアーキテクチャ「CloudMatrix 384 Supernode」は、384個のAscend 910Cチップを搭載し、NVIDIAのGB200 NVL72と比べて1.7倍の性能を実現しています。ただし消費電力は3.9倍と効率面での課題が残されています。すでに中国安徽省のデータセンターに設置され、中国製の推論モデルにも対応しています。

https://gigazine.net/news/20250418-huawei-ai-cloudmatrix-384-supernode/


### AIエージェントのみで構成されたSNSでは人間よりも誤情報の拡散が少ないことが研究で判明

【AI】研究者たちが「MOSAIC」というシミュレーションを使用して、AIだけのSNS環境を分析した結果が発表されました。人間が使用する実際のSNSと比較して、AIエージェントは誤情報の拡散を抑制する傾向が確認され、ファクトチェックも効果的に機能することが分かりました。また、投稿の人気度は内容そのものよりもシステムの設計に大きく影響されることも明らかになっています。

https://www.itmedia.co.jp/aiplus/articles/2504/18/news074.html


### AI研究者向けスキルアップ研修の資料を公開、コードレビューからライセンスまで幅広い技術テーマを網羅

【スキルアップ】AI Labが研究者向けに実施したスキルアップ研修の資料を公開しました。研修では、コードレビュー、ライセンス、図の作成など研究に必要な技術テーマを幅広く取り上げ、参加者から高い評価を得ています。オンライン・オフライン共に多くの参加があり、今後は参加者の技術レベルに応じた内容の最適化を目指します。

https://cyberagent.ai/blog/research/20228/


### Preferred Networksが日本語に特化したローカル実行可能な新テキスト埋め込みモデル「PLaMo-Embedding-1B」を公開

【AI】Preferred Networksが開発した日本語テキスト埋め込みモデル「PLaMo-Embedding-1B」が注目を集めています。ローカル環境で動作し、高性能な日本語テキストの埋め込みが可能な特徴を持ち、工場内でのデータ検索など、セキュアな環境での活用が期待されています。

https://dev.classmethod.jp/articles/shuntaka-try-plamo-embedding-1b/


### AIを活用した効率的な開発手法「Vibe Coding」の実践例：チャットAIとの対話から実装までの個人開発フロー

【つくってみた・やってみた】個人開発において、チャットAIを活用して要件定義から技術選定、実装までを効率的に進める「Vibe Coding」の実践方法が紹介されています。AIとの対話を通じて開発を進めながらも、Gitによるバージョン管理は人間が担当するなど、AIと人間の役割分担を明確にした開発スタイルが確立されています。

https://zenn.dev/yoshiko/articles/my-vibe-coding


### MITが開発、プログラミング言語の仕様に合わせてAIコード生成を最適化する新技術

【LLM新技術】MITの研究チームが、大規模言語モデルの出力を各プログラミング言語の仕様に自動的に適合させる新手法を開発しました。この技術により、小規模なLLMでも高精度なコード生成が可能となり、プログラミングアシスタントやデータ分析ツールなどへの幅広い応用が期待されています。

https://news.mit.edu/2025/making-ai-generated-code-more-accurate-0418


### スタンフォード大学の調査で中国のAIモデル性能が米国に追いつき、両国の技術格差が縮小傾向に

【LLMの評価】スタンフォード大学のAIインデックス調査によると、中国と米国の主要なAIモデルの性能差が大幅に縮小し、ほぼ同等の水準に達していることが明らかになりました。2024年の注目すべきAIモデル開発件数では、米国が40件、中国が15件を記録し、両国の主要企業が上位を占めています。

https://36kr.jp/341424/


### LLMを活用した実用的なAIエージェント開発のための12の原則が公開、本番環境での運用指針を提示

【AIエージェント】LLMを活用したAIエージェントを実際の本番環境で運用するための12の原則がGitHubで公開されました。この原則では、LLMの能力を最大限に活用しながらも、完全な自動化ではなく適切な人間の管理と制御の重要性を強調しています。

https://github.com/humanlayer/12-factor-agents


### AIエージェント開発の4要素：設計と実装における重要ポイントと課題を解説

【AIエージェント】AIエージェントの開発において、「instruction」「language model」「memory」「tools」という4つの重要な要素に基づいた設計・実装の知見が紹介されています。自律性と制御性のバランス、効果的なテスト方法、個性的なUXの実現など、実践的な課題とその対応策について解説されています。

https://zenn.dev/aishift/articles/6aa1540ea27fcd


### 小田急電鉄、IR業務効率化に向けて生成AI「exaBase IRアシスタント」を導入し業務DXを推進

【AIの活用】小田急電鉄がIR部門の業務効率化を目指し、生成AI「exaBase IRアシスタント」を導入することを発表しました。この取り組みにより、IR部門は定型的な業務から解放され、より付加価値の高い情報開示や投資家との対話に注力できる体制の構築を目指します。サービスを提供するExa Enterprise AIは、生成AI技術を通じて日本企業の生産性向上を支援しています。

https://robotstart.info/2025/04/18/odakyu-introduce-ai-ir-service.html


### 声優・梶裕貴さんのオリジナルキャラクター「そよぎ」がAI会話アプリ「HAPPY RAT」に登場、新たな物語体験を提供

【AIの活用】SpiralAIが提供する会話型友だちAIアプリ「HAPPY RAT」に、人気声優・梶裕貴さんが手がけるオリジナルコンテンツ「そよぎフラクタル」のキャラクターが登場することになりました。未来からやってきたネズミ型アンドロイド「そよぎ」との対話を通じて、ユーザーは新たな物語体験を楽しむことができます。

https://robotstart.info/2025/04/18/happy-rat-colabo-soyogi-fractal.html


### 生成AIが訴状から裁判の勝訴確率を予測し法的アドバイスを提供する新サービスをリーガルAIが開発

【AIの活用】リーガルAIが、訴状の内容をAIで分析し裁判の結果を予測する新サービスを開発しました。AIが法律や過去の判例データベースを活用して事実認定や法令解釈を行い、ユーザーに判決の見通しを提示することで、訴訟に関する意思決定をサポートします。

https://www.nikkei.com/article/DGXZQOUC10BSJ0Q5A410C2000000/


In [ ]:
# 特定のニュースをピックアップしたい場合
pickup_type = "day"
title_date = ""
title_name = "わくわくAIニュース"

# Notionデータベースからflag='true'でpickup_typeに合致するものを抽出
news_contents = main(pickup_type, title_date, title_name)

In [ ]:
# カテゴリごとに整理
NOTION_API_KEY = userdata.get('NOTION_API_KEY')
DATABASE_ID = userdata.get('PICKUP_DATABASE_KEY')

url = f'https://api.notion.com/v1/databases/{DATABASE_ID}/query'
headers = {
    'Notion-Version': '2022-06-28',
    'Authorization': f'Bearer {NOTION_API_KEY}',
    'Content-Type': 'application/json',
}

formatted_news = main_for_monthly_report(url, headers)
print(formatted_news)

### 【AIと人間の未来】
#### 〇 AIエージェントにホテルを予約させる実験から判明した衝撃の事実、AIはオンライン広告をどう見ているのか？ 【生成AI事件簿】構造化データを好むAIエージェントにバナー広告やブランド広告は不適か | JBpress (ジェイビープレス)
https://jbpress.ismedia.jp/articles/-/87753?page=2
従来のAPIによる連携に代わり、AIエージェントがブラウザを直接操作できるようになったことで、ウェブサイトの管理者には「GAIO（生成AI最適化）」と呼ばれる新たな対応が必要になってきている。また、この傾向を受けて、AIエージェントに効果的なオンライン広告の在り方について、構造化されたデータの重視やキーワードマッチングの重要性が指摘されている。

#### 〇 再犯防止と更生支援のために開発されたアルゴリズム「TIGER」が受刑者の仮釈放資格を判断する唯一の基準となってしまっている
https://gigazine.net/news/20250415-tiger-algorithm/
ルイジアナ州は、2014年に開発した再犯防止アルゴリズム「TIGER」を使って、受刑者の仮釈放を判断するようになりました。しかし、TIGERスコアが唯一の基準となり、受刑者の更生状況にかかわらず仮釈放を拒否することが問題視されています。専門家からは、過去の情報にのみ依存するTIGERの問題点が指摘されています。

#### 〇 “AIが95％ 芥川賞作家が5％書いた小説” 雑誌に掲載 | NHK
https://www3.nhk.or.jp/news/html/20250325/k10014759151000.html
芥川賞作家の九段理江さんと生成AIが共同で執筆した小説「影の雨」が、雑誌に掲載されました。この作品は、人間の記憶や感情の痕跡を辿りながら、「感情とは何のためにあるのか」を探究するという内容の短編小説です。九段さんは全体の5％を担当し、冒頭と文末の修正を行った一方で、95％はAIが執筆したものです。九段さんは、この経験を通して、人間がフィクションを想像する本質的な意味について考え直す機会となったと述べています。

#### 〇 ChatGPTの使用が感情的な幸福にどのように影響するかに関する初の研